Get images and save them independently as 2D slices

Dependencies

In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons

import scipy

import SimpleITK as sitk
reader = sitk.ImageFileReader()
reader.SetImageIO("MetaImageIO")

import numpy as np

import os

import pathlib

from natsort import natsorted

#Set GPU/Cuda Device to run model on
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

np.random.seed(46)

Using cuda device


Dataset Directories

In [2]:
#Toy Dataset Directory from Spider | Grand Challenge 
dummy_train_img_dir = pathlib.Path(r"spider_toy_dset/dummy_train_images")
dummy_train_label_dir = pathlib.Path(r"spider_toy_dset/dummy_train_labels")
dummy_test_img_dir = pathlib.Path(r"spider_toy_dset/dummy_test_images")
dummy_test_label_dir= pathlib.Path(r"spider_toy_dset/dummy_test_labels")

Image Class, Returns 3D Array

In [3]:
from transforms import mri_transforms


#TODO add bool image label for interp 
class Mri:
    def __init__(self, path):
        mri_mha = sitk.ReadImage(path, imageIO = "MetaImageIO") #explicitly setting ioreader just in case
        
       #TODO use sitk permute to change dimensions to 2 0 1 to help with going through slices for interpolating in 2d
        if(mri_mha.GetSize()[0] > mri_mha.GetSize()[1] or mri_mha.GetSize()[0] > mri_mha.GetSize()[2]): #if z axis isn't first
          mri_mha = sitk.PermuteAxes(mri_mha, (2, 0, 1))
          
        #get 3d array from mri
        mri_a = np.array(sitk.GetArrayFromImage(mri_mha)) #mri_array

        #transpose array to format z x y or z y x 
        if(mri_a.shape[0] > mri_a.shape[1] or mri_a.shape[0] > mri_a.shape[2]): #if z axis isn't first
          mri_a = np.transpose(mri_a, (2, 0, 1))

        '''
        if(mri_a.shape[2] > mri_a.shape[1]): #bring images to vertical orientation
          mri_a = np.transpose(mri_a, (0, 2, 1))
        '''
      
        mri_a_float32 = mri_a.astype(dtype = np.float32)
        #TODO: set bounds to [-1000, 2000] https://en.wikipedia.org/wiki/Hounsfield_scale
        self.hu_a = mri_a_float32

Sort Directories

In [4]:
#get lists from directories

#toy dset 

label_train_dir_list = os.listdir(dummy_train_label_dir)
image_train_dir_list = os.listdir(dummy_train_img_dir) 

#local dset
'''
image_dir_list = os.listdir(local_img_idr)
label_dir_list = os.listdir(local_label_dir)
'''
#sort lists
image_train_dir_list = natsorted(image_train_dir_list)
label_train_dir_list = natsorted(label_train_dir_list)
#empty lists to hold x and y dimensions of images
row_list = []
col_list = []

#dirlen = len(os.listdir(dummy_train_label_dir))
dirlen = len(os.listdir(dummy_train_label_dir))

print(dirlen)

#print(local_label_idr)


44


Testing on single MRI

In [5]:
from transforms import array_transforms

dummy_train_img_slice_dir = pathlib.Path(r"spider_toy_dset_slices/dummy_train_img_slices")
dummy_train_label_slice_dir = pathlib.Path(r"spider_toy_dset_slices/dummy_train_label_slices")
dummy_test_img_slice_dir = pathlib.Path(r"spider_toy_dset_slices/dummy_test_img_slices")
dummy_test_label_slice_dir= pathlib.Path(r"spider_toy_dset_slices/dummy_test_label_slices")

img_path = dummy_train_img_dir.joinpath(image_train_dir_list[0])
test_mri = Mri(img_path)
print(test_mri.hu_a.shape)

#array_transforms.extract_slices(test_mri.hu_a, image_dir_list[0], dummy_train_img_slice_dir)



(50, 578, 448)


Remove slices with 0 label value and save the remaining slices in a new directory | TOY TRAIN SET 

In [6]:
#train directories images to slices 
from transforms import array_transforms

for idx in range(0, dirlen):
  #print("dirlen", dirlen)
  
 #toy dset 
  
  img_path = dummy_train_img_dir.joinpath(image_train_dir_list[idx])
  label_path = dummy_train_label_dir.joinpath(label_train_dir_list[idx])#first part before joinpath is pathlib.Path, second part is the directory of the file 

  #print(image_train_dir_list[idx])
  #print(type(image_train_dir_list[idx]))

  '''
  img_path = local_img_idr.joinpath(image_dir_list[idx])
  label_path = local_label_dir.joinpath(label_dir_list[idx]) #first part before joinpath is pathlib.Path, second part is the directory of the file 
  '''
  image = Mri(img_path)
  label = Mri(label_path)
  

  #print("before", label.hu_a.shape)
  #remove empty slices
  image_a, label_a = array_transforms.remove_empty_slices(image.hu_a, label.hu_a)

  array_transforms.extract_slices(image_a, image_train_dir_list[idx], dummy_train_img_slice_dir) 
  array_transforms.extract_slices(label_a, label_train_dir_list[idx], dummy_train_label_slice_dir) 

  #print("label after: ", label_a.shape)
  


  






row max: 899
col max: 514


Remove slices with 0 label value and save the remaining slices in a new directory | TOY TEST SET 

In [7]:
label_test_dir_list = os.listdir(dummy_test_label_dir)
image_test_dir_list = os.listdir(dummy_test_img_dir) 

#local dset
'''
image_dir_list = os.listdir(local_img_idr)
label_dir_list = os.listdir(local_label_dir)
'''
#sort lists
image_train_dir_list = natsorted(image_test_dir_list)
label_train_dir_list = natsorted(label_test_dir_list)

dirlen = len(os.listdir(dummy_test_label_dir))

print(dirlen)

#train directories images to slices 
from transforms import array_transforms

for idx in range(0, dirlen):
  #print("dirlen", dirlen)
  
 #toy dset 
  
  img_path = dummy_test_img_dir.joinpath(image_test_dir_list[idx])
  label_path = dummy_test_label_dir.joinpath(label_test_dir_list[idx])#first part before joinpath is pathlib.Path, second part is the directory of the file 

  #print(image_train_dir_list[idx])
  #print(type(image_train_dir_list[idx]))

  '''
  img_path = local_img_idr.joinpath(image_dir_list[idx])
  label_path = local_label_dir.joinpath(label_dir_list[idx]) #first part before joinpath is pathlib.Path, second part is the directory of the file 
  '''
  image = Mri(img_path)
  label = Mri(label_path)
  

  #print("before", label.hu_a.shape)
  #remove empty slices
  image_a, label_a = array_transforms.remove_empty_slices(image.hu_a, label.hu_a)

  array_transforms.extract_slices(image_a, image_test_dir_list[idx], dummy_test_img_slice_dir) 
  array_transforms.extract_slices(label_a, label_test_dir_list[idx], dummy_test_label_slice_dir) 

  #print("label after: ", label_a.shape)
  


  






10
